In [13]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
import tensorflow as tf
from keras import layers, models, losses, metrics, regularizers, optimizers
from sklearn.model_selection import train_test_split

In [ ]:
CHARSET = "0123456789abcdefghijklmnopqrstuvwxyz"

In [ ]:
def resize_and_pad_image(img, target_size):
    height, width = img.shape[0], img.shape[1]

    scale = target_size / max(height, width)
    new_height = int(height * scale)
    new_width = int(width * scale)

    resized = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    pad_height = target_size - new_height
    pad_width = target_size - new_width
    top = pad_height // 2
    bottom = pad_height - top
    left = pad_width // 2
    right = pad_width - left

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

    return padded

In [ ]:
def load_segmented_images(folder_path):
    X_list, Y_list = [], []
    test_dict = dict()
    captcha_names = os.listdir(folder_path)
    train_names, test_names = train_test_split(captcha_names, test_size=0.1, random_state=42)
    for captcha_name in captcha_names:
        captcha_dir = os.path.join(folder_path, captcha_name)
        if not os.path.isdir(captcha_dir) or len(captcha_name) == 0:
            continue
        image_files = [fname for fname in os.listdir(captcha_dir)
                if fname.lower().endswith(".png") and fname.startswith("char_")]
        for filename in image_files:
            try:
                char_index = int(filename.split("_")[1].split(".")[0])
            except Exception:
                continue
            if char_index >= len(captcha_name):
                continue
            label_char = captcha_name[char_index]
            if label_char not in CHARSET:
                continue
            label_idx = CHARSET.index(label_char)
            img_path = os.path.join(captcha_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            
            img = resize_and_pad_image(img, 28)
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=-1)
            label = np.zeros(len(CHARSET), dtype=np.float32)
            label[label_idx] = 1.0

            if captcha_name in train_names:
                X_list.append(img)
                Y_list.append(label)

            else:
                test_dict.setdefault(captcha_name, ([], []))
                test_dict[captcha_name][0].append(img)
                test_dict[captcha_name][1].append(label)
    return X_list, Y_list, test_dict

In [ ]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

segment_folder = "segmented_new"
X_list, Y_list, test_dict = load_segmented_images(segment_folder)

X_train, X_val, Y_train, Y_val = train_test_split(X_list, Y_list, test_size=0.1, random_state=42)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print(f"Loaded {len(X_train)} training characters, {len(X_val)} validation characters and {len(test_dict)} test strings.")

Loaded 29664 training characters, 3297 validation characters and 614 test strings.


In [18]:
from PIL import Image

display(Image.fromarray(X_val[1000].reshape(28, 28) * 255).convert("L"))

In [ ]:
def build_model(num_classes=36):
    inputs = layers.Input(shape=(28, 28, 1))
    data_augmentation = tf.keras.Sequential([
        layers.RandomRotation(0.1),      
        layers.RandomZoom(0.1),             
        layers.RandomTranslation(0.1, 0.1),  
        layers.RandomContrast(0.2),       
        layers.GaussianNoise(0.1)       
    ])

    x = data_augmentation(inputs)
    
    x = layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    filters_list = [64, 128, 256]
    for i, filters in enumerate(filters_list):
        shortcut = x
        
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)
        
        x = layers.Conv2D(filters, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Conv2D(filters, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.add([shortcut, x])
    
        if i < len(filters_list) - 1:
            x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    
    model.compile(
        optimizer=optimizer,
        loss=losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=[metrics.CategoricalAccuracy()])
    
    return model

In [ ]:

model = build_model(num_classes=len(CHARSET))
model.summary()

callbacks = [
tf.keras.callbacks.ModelCheckpoint(
    'best_model.keras', save_best_only=True, monitor='val_categorical_accuracy'),
tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
tf.keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy', patience=10, 
    restore_best_weights=True, mode='max', baseline=0.4),
tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

class_counts = np.sum(Y_train, axis=0)
class_weights = {i: 1.0 / (count + 1e-5) for i, count in enumerate(class_counts)}
class_weights = {k: v / sum(class_weights.values()) * len(CHARSET) for k, v in class_weights.items()}

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 28, 28, 1) │          0 │ input_layer_1[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 28, 28,    │        640 │ sequential[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        256 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 14, 14,    │     36,928 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │        256 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 14, 14,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 14, 14,    │     36,928 │ dropout_4[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 14, 14,    │          0 │ max_pooling2d_3[… │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 7, 7, 64)  │          0 │ add_3[0][0]       │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 7, 7, 128) │     73,856 │ max_pooling2d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 128) │        512 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 7, 7, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 7, 7, 128) │      8,320 │ max_pooling2d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 7, 7, 128) │    147,584 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 128) │        512 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 1,380,004 (5.26 MB)

 Trainable params: 1,376,292 (5.25 MB)

 Non-trainable params: 3,712 (14.50 KB)

In [21]:
history = model.fit(
X_train, Y_train,
epochs=100,
batch_size=128,
validation_data=(X_val, Y_val),
callbacks=callbacks,
class_weight=class_weights)

Epoch 1/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - categorical_accuracy: 0.1770 - loss: 3.6704

232/232 ━━━━━━━━━━━━━━━━━━━━ 32s 116ms/step - categorical_accuracy: 0.1775 - loss: 3.6677 - val_categorical_accuracy: 0.0567 - val_loss: 3.7064 - learning_rate: 0.0010
Epoch 2/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - categorical_accuracy: 0.5190 - loss: 2.1157

232/232 ━━━━━━━━━━━━━━━━━━━━ 26s 114ms/step - categorical_accuracy: 0.5191 - loss: 2.1153 - val_categorical_accuracy: 0.6460 - val_loss: 1.7313 - learning_rate: 0.0010
Epoch 3/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - categorical_accuracy: 0.6332 - loss: 1.7689

232/232 ━━━━━━━━━━━━━━━━━━━━ 26s 113ms/step - categorical_accuracy: 0.6333 - loss: 1.7688 - val_categorical_accuracy: 0.7367 - val_loss: 1.5284 - learning_rate: 0.0010
Epoch 4/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 25s 109ms/step - categorical_accuracy: 0.6792 - loss: 1.6346 - val_categorical_accuracy: 0.7328 - val_loss: 1.5591 - learning_rate: 0.0010
Epoch 5/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - categorical_accuracy: 0.7068 - loss: 1.5459

232/232 ━━━━━━━━━━━━━━━━━━━━ 26s 110ms/step - categorical_accuracy: 0.7068 - loss: 1.5458 - val_categorical_accuracy: 0.7734 - val_loss: 1.3939 - learning_rate: 0.0010
Epoch 6/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - categorical_accuracy: 0.7268 - loss: 1.4874

...

232/232 ━━━━━━━━━━━━━━━━━━━━ 27s 115ms/step - categorical_accuracy: 0.8763 - loss: 1.0258 - val_categorical_accuracy: 0.9008 - val_loss: 0.9749 - learning_rate: 2.5000e-04
Epoch 43/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - categorical_accuracy: 0.8768 - loss: 1.0192

232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 119ms/step - categorical_accuracy: 0.8768 - loss: 1.0192 - val_categorical_accuracy: 0.9020 - val_loss: 0.9768 - learning_rate: 2.5000e-04
Epoch 44/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 120ms/step - categorical_accuracy: 0.8806 - loss: 1.0115 - val_categorical_accuracy: 0.8993 - val_loss: 0.9740 - learning_rate: 2.5000e-04
Epoch 45/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - categorical_accuracy: 0.8816 - loss: 1.0177

232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 120ms/step - categorical_accuracy: 0.8816 - loss: 1.0177 - val_categorical_accuracy: 0.9026 - val_loss: 0.9700 - learning_rate: 2.5000e-04
Epoch 46/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - categorical_accuracy: 0.8790 - loss: 1.0153

232/232 ━━━━━━━━━━━━━━━━━━━━ 29s 125ms/step - categorical_accuracy: 0.8790 - loss: 1.0153 - val_categorical_accuracy: 0.9051 - val_loss: 0.9724 - learning_rate: 2.5000e-04
Epoch 47/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 27s 118ms/step - categorical_accuracy: 0.8807 - loss: 1.0109 - val_categorical_accuracy: 0.9051 - val_loss: 0.9682 - learning_rate: 2.5000e-04
Epoch 48/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - categorical_accuracy: 0.8827 - loss: 1.0055

232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 120ms/step - categorical_accuracy: 0.8827 - loss: 1.0055 - val_categorical_accuracy: 0.9072 - val_loss: 0.9679 - learning_rate: 2.5000e-04
Epoch 49/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 27s 117ms/step - categorical_accuracy: 0.8816 - loss: 1.0072 - val_categorical_accuracy: 0.9032 - val_loss: 0.9656 - learning_rate: 2.5000e-04
Epoch 50/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 119ms/step - categorical_accuracy: 0.8802 - loss: 1.0097 - val_categorical_accuracy: 0.9042 - val_loss: 0.9659 - learning_rate: 2.5000e-04
Epoch 51/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 122ms/step - categorical_accuracy: 0.8849 - loss: 1.0013 - val_categorical_accuracy: 0.9048 - val_loss: 0.9688 - learning_rate: 2.5000e-04
Epoch 52/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 27s 116ms/step - categorical_accuracy: 0.8854 - loss: 0.9966 - val_categorical_accuracy: 0.9045 - val_loss: 0.9652 - learning_rate: 1.2500e-04
Epoch 53/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 27s 116ms/step - categorical_accuracy: 0.8890 

In [ ]:
def evaluate_captcha_with_per_class(model, test_dict, charset):    
    total_chars = 0
    correct_chars = 0
    total_strings = len(test_dict)
    correct_strings = 0
    
    class_correct = {i: 0 for i in range(len(charset))}
    class_total   = {i: 0 for i in range(len(charset))}
    
    tqdm_bar = tqdm(total=total_strings, desc="Evaluating CAPTCHA strings")
    
    for captcha_name, (X_data, Y_data) in test_dict.items():
        total_chars += len(X_data)
        
        pred = model.predict(np.array(X_data), verbose=0)
        pred_labels = np.argmax(pred, axis=1)
        true_labels = np.argmax(np.array(Y_data), axis=1)

        is_string_correct = True
        
        for i in range(len(pred_labels)):
            class_total[true_labels[i]] += 1  

            if pred_labels[i] == true_labels[i]:
                correct_chars += 1
                class_correct[true_labels[i]] += 1
            else:
                is_string_correct = False
        
        if is_string_correct:
            correct_strings += 1
        
        tqdm_bar.update(1)
    
    tqdm_bar.close()

    char_accuracy = (correct_chars / total_chars) * 100
    string_accuracy = (correct_strings / total_strings) * 100
    
    print(f"\nOverall Character Accuracy: {char_accuracy:.2f}%")
    print(f"Overall String Accuracy: {string_accuracy:.2f}%")
    
    per_class_accuracy = {}
    for i in range(len(charset)):
        if class_total[i] > 0:
            acc = (class_correct[i] / class_total[i]) * 100
        else:
            acc = 0  
        per_class_accuracy[charset[i]] = acc

    print("\nPer-Class Accuracies:")
    for char, acc in sorted(per_class_accuracy.items(), key=lambda x: x[1]):
        print(f"Character '{char}': {acc:.2f}%")
    
    worst_char, worst_acc = min(per_class_accuracy.items(), key=lambda x: x[1])
    print(f"\nLowest Performing Character: '{worst_char}' with accuracy: {worst_acc:.2f}%")



In [ ]:
evaluate_captcha_with_per_class(model, test_dict, CHARSET)

Evaluating CAPTCHA strings: 100%|██████████| 614/614 [00:35<00:00, 17.16it/s]


Overall Character Accuracy: 89.68%
Overall String Accuracy: 59.45%

Per-Class Accuracies:
Character 'o': 52.13%
Character 'i': 67.77%
Character '0': 68.29%
Character '1': 70.64%
Character 'g': 82.42%
Character 'l': 82.50%
Character 'q': 85.57%
Character 's': 87.50%
Character 'u': 87.88%
Character 'j': 89.52%
Character 'a': 90.12%
Character '7': 91.00%
Character 'y': 91.11%
Character '8': 91.26%
Character 'z': 91.46%
Character 'h': 92.86%
Character 'e': 92.93%
Character 'v': 93.42%
Character '9': 93.46%
Character 'd': 94.02%
Character 'w': 94.19%
Character 'r': 94.29%
Character '2': 94.39%
Character 'm': 95.00%
Character 't': 95.00%
Character '6': 95.61%
Character 'x': 95.65%
Character 'n': 95.83%
Character 'f': 96.00%
Character 'p': 96.08%
Character '5': 96.33%
Character 'b': 97.00%
Character '4': 97.27%
Character 'c': 97.27%
Character 'k': 99.12%
Character '3': 100.00%

Lowest Performing Character: 'o' with accuracy: 52.13%
